**Library import**

In [2]:
%matplotlib qt 
# Active l'affichage dynamique sur le notebook
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keyboard # Pas sûr que vous ayez la librairie d'installer sur vos PC
import time
import math as m
import random

**Creation of dataframe from a text file converted into a CSV**

In [16]:
# # Read data from the text file
# with open('dynamic_B1_A2_0.txt', 'r') as file:
#     lines = file.readlines()

# # Process data
# data = []
# for line in lines:
#     if line.strip() and not line.startswith('*'):
#         parts = line.strip().split()
#         tracker = parts[0].strip(':')
#         id = parts[1]
#         x = parts[2]
#         y = parts[3]
#         z = parts[4]
#         timestamp = parts[5]
#         data.append([tracker, id, x, y, z, timestamp])

# # Write data to a CSV file
# with open('output.csv', 'w', newline='') as csvfile:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerow(['Tracker', 'ID', 'X', 'Y', 'Z', 'Timestamp'])
#     csvwriter.writerows(data)

# print("Data exported in CSV")

# data = pd.read_csv("output.csv")
# data_n = data.drop(columns=["Tracker", "ID"])

Data exported in CSV


In [4]:
data = pd.read_csv("dynamic_B2_O5_A2_A1_O1_B1_B2_0_0.csv")
data_n = data.drop(columns=["Tracker", "ID"])

**Add disturbance in data**

In [48]:
# We had some pertubation in data

probability = 0.1
masque = np.random.rand(len(data_n)) < probability
probability2 = 0.5
masque2 = np.random.rand(len(data_n)) < probability
# Multiplie les valeurs de la colonne 'X' et 'Y' par 10 là où le masque est True
data_n.loc[masque, 'X'] *= -2
data_n.loc[masque, 'Y'] *= -2
data_n.loc[masque, 'X'] *= 1.5
data_n.loc[masque, 'Y'] *= 1.2

**Exclusive radius sorting**

In [93]:
def exclusive_radius(data_x,data_y,R=0.95):  # R en mètre
    data_x_ = []
    prec_ecart_x = 0
    prec_ecart_y = 0
    for i in range(1,len(data_x)):
        if(i>3):
            if m.sqrt((data_x[i]-data_x[i-1])**2+(data_y[i]-data_y[i-1])**2) > R:
                data_x[i] = data_x[i-1] + prec_ecart_x
                data_y[i] = data_y[i-1] + prec_ecart_y
        if(i>1 and (m.sqrt((data_x[i]-data_x[i-1])**2+(data_y[i]-data_y[i-1])**2) <= R)):
            prec_ecart_x = data_x[i]-data_x[i-1]
            prec_ecart_y = data_y[i]-data_y[i-1]
    return

data_n_filtered = data_n.copy() # dataframe sans les valeurs 

exclusive_radius(data_n_filtered['X'],data_n_filtered['Y'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
#print(data_n_filtered['X'],data_n_filtered['Y'])

**EMA sorting**

In [94]:
# Partie filtrage
# Define the EMA function, if the velocity is high alpha should be close to 1
def alpha_value(x1,x2,y1,y2):
    velocity = m.sqrt((x2-x1)**2+(y2-y1)**2)/0.2
    alpha = velocity/4.2
    if alpha > 1:
        return 1
    elif alpha >= 0.05:
        return alpha
    else:    
        return 0.05
    
    
    
def exponential_moving_average(data_1,data_2):
    ema = [data_1[0]]
    alpha = alpha_value(data_1[0],data_1[1],data_2[0],data_2[1])
    for i in range(1,len(data_1)):
        alpha = alpha_value(data_1[i-1],data_1[i],data_2[i-1],data_2[i])
        ema.append(alpha * data_1[i] + (1 - alpha) * ema[-1])
    return ema

# Calculate EMA for the 'X' and 'Y' columns
data_n_filtered['EMA_X'] = exponential_moving_average(data_n_filtered['X'],data_n_filtered['Y'])
data_n_filtered['EMA_Y'] = exponential_moving_average(data_n_filtered['Y'],data_n_filtered['X'])

**Displaying raw data**

In [95]:

fig,ax= plt.subplots()
line, = ax.plot([], [], 'ro-')

# Set plot limits
# ax.set_xlim(min(data_n['X']) - 1, max(data_n['X']) + 1)
# ax.set_ylim(min(data_n['Y']) - 1, max(data_n['Y']) + 1)
ax.set_xlim(-2,18)
ax.set_ylim(-2,18)

for i in range(data_n.shape[0]):
    x = data_n['X'][i]
    y = data_n['Y'][i]
    plt.title('Raw Data Plot')
    plt.scatter(x,y,color = 'red',s = 15)
    plt.pause(0.0001)
    if keyboard.is_pressed('q'):
        plt.close()
        break
        
plt.show()

**Diplaying sorted data**

In [96]:
# Animation function
fig,ax= plt.subplots()
line, = ax.plot([], [], 'ro-')

# Set plot limits
# ax.set_xlim(min(data_n['X']) - 1, max(data_n['X']) + 1)
# ax.set_ylim(min(data_n['Y']) - 1, max(data_n['Y']) + 1)
ax.set_xlim(-2,18)
ax.set_ylim(-2,18)

for i in range(data_n_filtered.shape[0]):
    x_fil = data_n_filtered['X'][i]
    y_fil = data_n_filtered['Y'][i]
    #x_ema = data_n_filtered['EMA_X'][i]
    #y_ema = data_n_filtered['EMA_Y'][i]
    plt.title('sorted Data Plot')
    plt.scatter(x_fil,y_fil, color = 'green', s = 10)
    #plt.scatter(x_ema,y_ema, color = 'blue',s = 10)
    plt.pause(0.0001)
    if keyboard.is_pressed('q'):
        plt.close()
        break
        
plt.show()